<a href="https://colab.research.google.com/github/Tee-A/AIRLINES-AIRCRAFTS_TAXI-OUT-TIME-PREDICTION/blob/master/PyCaret_Catboost_Exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing and Installing Necessary libraries and Packages

In [0]:
!pip install pycaret

     |████████████████████████████████| 194kB 3.4MB/s 
     |████████████████████████████████| 1.2MB 27.7MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 1.6MB 34.5MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 7.0MB 23.3MB/s 
     |████████████████████████████████| 3.0MB 30.9MB/s 
     |████████████████████████████████| 266kB 43.5MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 133kB 41.7MB/s 
     |████████████████████████████████| 389kB 37.5MB/s 
     |████████████████████████████████| 63.9MB 67kB/s 
     |████████████████████████████████| 235kB 40.9MB/s 
     |████████████████████████████████| 552kB 33.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 6.2MB 32.7MB/s 
     |████████████████████████████████| 2.1MB 31.3MB/s 
   

In [0]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
train=pd.read_csv("/content/drive/My Drive/Data_Science/Zindi_Competitions/Zindi_Airquo/Airquo_resources/Train.csv")
test=pd.read_csv("/content/drive/My Drive/Data_Science/Zindi_Competitions/Zindi_Airquo/Airquo_resources/Test.csv")

In [0]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

In [0]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [0]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

In [0]:
for x in range(25,121):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

In [0]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.90it/s]


In [0]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:24<00:00,  4.05s/it]


In [0]:
data.drop(features,1,inplace=True)

In [0]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [0]:
# Creating columns from missing values columns
def create_na_columns(x):
    if (pd.isnull(x)):
        return 1
    else: 
        return 0
features=train.columns
for feature in features : 
    train[feature + "_na"]=train[feature].apply(create_na_columns)
    test[feature + "_na"]=test[feature].apply(create_na_columns)    

In [0]:
train = train.fillna(train.mean())

In [0]:
test = test.fillna(test.mean())

In [0]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [0]:
lab.fit(train["location"])
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
# standize the matrix for training data
scaler = StandardScaler()
train1 = scaler.fit_transform(train.drop(['ID','target',],1))

In [0]:
test_scaler = scaler.transform(test.drop(["ID","target",],axis = 1))

# Further EDA and Feature Eng with PyCaret

In [0]:
dataset = train.drop('ID',1)

In [0]:
data = dataset
data_unseen = test.drop(['ID','target'],1)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))

Data for Modeling: (15539, 1241)
Unseen Data For Predictions (5035, 1240)


In [0]:
from pycaret.regression import *

In [0]:
exp_reg102 = setup(data = data, target = 'target', session_id=123,
                  normalize = True, transformation = True,
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  bin_numeric_features = ['max_wind_dir','min_wind_dir', 'mean_wind_dir', 'std_wind_dir', 'var_wind_dir',
                                          'median_wind_dir', 'ptp_wind_dir','max_atmos_press', 'min_atmos_press',
                                          'mean_atmos_press', 'std_atmos_press', 'var_atmos_press','median_atmos_press', 'ptp_atmos_press'])

 
Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(15539, 1241)"
4,Missing Values,False
5,Numeric Features,618
6,Categorical Features,622
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [0]:
exp_reg102 = setup(data = data, target = 'target', session_id=123,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  bin_numeric_features = ['max_wind_dir','min_wind_dir', 'mean_wind_dir', 'std_wind_dir', 'var_wind_dir',
                                          'median_wind_dir', 'ptp_wind_dir',])

In [0]:
exp_reg102 = setup(data = data, target = 'target', session_id=123,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95,)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Transform Target,True
2,Transform Target Method,box-cox
3,Original Data,"(15539, 770)"
4,Missing Values,True
5,Numeric Features,768
6,Categorical Features,1
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


# Creating Models and Tunning

### Catboost Regressor

In [0]:
tuned_catboost  = tune_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,16.3658,626.5339,25.0307,0.6132,0.3748,0.3442
1,16.3412,750.6641,27.3982,0.6200,0.3706,0.3348
2,16.4325,730.2626,27.0234,0.6259,0.3584,0.3197
3,15.9545,627.4026,25.0480,0.6186,0.3688,0.3339
4,15.5236,666.3711,25.8142,0.6244,0.3804,0.3488
5,17.0262,695.2071,26.3668,0.6419,0.3894,0.3622
6,16.3331,632.6323,25.1522,0.6507,0.3967,0.3806
7,17.0268,726.7496,26.9583,0.6468,0.3912,0.3740
8,16.2045,703.2222,26.5183,0.6498,0.3795,0.3521
9,15.7349,715.6852,26.7523,0.6003,0.3781,0.3429


In [0]:
tuned_catboost  = tune_model('catboost',fold=5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.9644,740.4979,27.2121,0.5883,0.3464,0.2821
1,16.0717,776.9932,27.8746,0.5683,0.3448,0.2744
2,15.7622,736.4023,27.1367,0.6041,0.3566,0.2914
3,16.1627,716.3967,26.7656,0.6211,0.3665,0.3157
4,16.0279,860.0680,29.3269,0.5575,0.3590,0.2945
Mean,15.9978,766.0716,27.6632,0.5879,0.3546,0.2916
SD,0.1343,50.9048,0.9054,0.0231,0.0081,0.0140


In [0]:
tuned_catboost  = tune_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.6944,664.2487,25.7730,0.5899,0.3452,0.2803
1,16.4718,865.4798,29.4190,0.5619,0.3486,0.2833
2,16.1191,860.0548,29.3267,0.5595,0.3394,0.2627
3,15.4075,683.3382,26.1407,0.5846,0.3433,0.2786
4,15.1697,720.7785,26.8473,0.5937,0.3517,0.2921
5,16.3740,748.8805,27.3657,0.6142,0.3597,0.2911
6,15.6444,676.3754,26.0072,0.6266,0.3565,0.3054
7,16.4731,798.5502,28.2586,0.6119,0.3605,0.3032
8,15.8988,791.4855,28.1334,0.6058,0.3547,0.2933
9,15.3638,798.4573,28.2570,0.5540,0.3507,0.2850


In [0]:
tuned_catboost  = tune_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,15.6803,663.2514,25.7537,0.5905,0.3447,0.2808
1,16.6072,879.2746,29.6526,0.5549,0.3565,0.2897
2,16.3725,864.2566,29.3982,0.5573,0.3427,0.2686
3,15.4330,679.9898,26.0766,0.5866,0.3420,0.2784
4,15.1898,714.3504,26.7273,0.5973,0.3499,0.2916
5,16.5068,765.7050,27.6714,0.6056,0.3626,0.2939
6,15.6202,675.2976,25.9865,0.6271,0.3569,0.3043
7,16.5705,796.9391,28.2301,0.6127,0.3634,0.3075
8,15.8234,799.2158,28.2704,0.6020,0.3534,0.2899
9,15.3672,797.3042,28.2366,0.5547,0.3515,0.2855


In [0]:
predict_model(tuned_catboost);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,16.2012,672.325,25.9292,0.5919,0.3766,0.3425


In [0]:
predict_model(tuned_catboost);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,15.2454,697.6961,26.4139,0.5765,0.3413,0.2746


In [0]:
predict_model(tuned_catboost);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,15.5193,715.6688,26.752,0.5656,0.3469,0.281


In [0]:
predict_model(tuned_catboost);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,15.5854,720.4007,26.8403,0.5627,0.3481,0.2828


# Finalizing Model

In [0]:
final_catboost = finalize_model(tuned_catboost)

In [0]:
unseen_predictions = predict_model(final_catboost, data=test.drop(['ID','target'],axis=1))
unseen_predictions.head()

,location,newtemp25,newprecip25,newrel_humidity25,newwind_dir25,windspeed25,atmospherepressure25,newtemp26,newprecip26,newrel_humidity26,newwind_dir26,windspeed26,atmospherepressure26,newtemp27,newprecip27,newrel_humidity27,newwind_dir27,windspeed27,atmospherepressure27,newtemp28,newprecip28,newrel_humidity28,newwind_dir28,windspeed28,atmospherepressure28,newtemp29,newprecip29,newrel_humidity29,newwind_dir29,windspeed29,atmospherepressure29,newtemp30,newprecip30,newrel_humidity30,newwind_dir30,windspeed30,atmospherepressure30,newtemp31,newprecip31,newrel_humidity31,...,std_temp_na,var_temp_na,median_temp_na,ptp_temp_na,max_precip_na,min_precip_na,mean_precip_na,std_precip_na,var_precip_na,median_precip_na,ptp_precip_na,max_rel_humidity_na,min_rel_humidity_na,mean_rel_humidity_na,std_rel_humidity_na,var_rel_humidity_na,median_rel_humidity_na,ptp_rel_humidity_na,max_wind_dir_na,min_wind_dir_na,mean_wind_dir_na,std_wind_dir_na,var_wind_dir_na,median_wind_dir_na,ptp_wind_dir_na,max_wind_spd_na,min_wind_spd_na,mean_wind_spd_na,std_wind_spd_na,var_wind_spd_na,median_wind_spd_na,ptp_wind_spd_na,max_atmos_press_na,min_atmos_press_na,mean_atmos_press_na,std_atmos_press_na,var_atmos_press_na,median_atmos_press_na,ptp_atmos_press_na,Label
0,2,21.208333,0.0,0.852500,190.278481,0.305833,87.533333,20.966667,0.0,0.893000,250.096677,0.278333,87.619167,20.691667,0.000,0.890917,241.577786,0.240000,87.700000,20.658333,0.000,0.858000,147.193406,0.356667,87.785833,20.700000,0.0,0.846750,183.214287,0.171667,87.820000,20.883333,0.0,0.845167,334.938191,0.574167,87.776667,20.441667,0.068,0.884083,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,147.2490
1,2,19.191667,0.0,0.909667,324.148034,0.813333,88.095000,20.250000,0.0,0.885583,326.496489,1.043333,88.125833,22.616667,0.000,0.797583,334.258793,0.975833,88.149167,24.333333,0.000,0.735083,331.914632,0.842500,88.151667,25.208333,0.0,0.663667,338.983475,0.979167,88.081667,26.000000,0.0,0.654417,329.855543,1.208333,87.991667,23.650000,0.000,0.674000,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84.4121
2,3,24.441667,0.0,0.859417,157.552130,1.605000,90.329167,23.458333,0.0,0.866250,154.860045,0.924167,90.368333,23.150000,0.000,0.871917,116.744172,0.420833,90.449167,22.366667,0.000,0.899583,164.772548,0.400833,90.508333,21.708333,0.0,0.914167,33.232804,0.294167,90.582500,21.816667,0.0,0.910500,338.069028,0.372500,90.664167,21.708333,0.000,0.914083,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32.5173
3,2,21.241667,0.0,0.871917,245.356443,0.453333,87.930000,22.908333,0.0,0.830500,227.876639,0.696667,87.939167,24.291667,0.425,0.785917,180.216419,1.270000,87.927500,23.858333,0.102,0.816083,174.482142,1.348333,87.829167,24.941667,0.0,0.713917,171.162789,1.261667,87.755000,26.241667,0.0,0.689417,173.633083,1.340833,87.646667,26.050000,0.000,0.687583,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68.8173
4,3,17.600000,0.0,0.866667,305.320114,0.620000,90.519167,20.141667,0.0,0.879167,45.007555,0.458333,90.565833,22.791667,0.000,0.842500,141.916191,0.865000,90.588333,24.958333,0.000,0.725000,118.422088,1.032500,90.559167,26.683333,0.0,0.657500,87.415779,2.045000,90.510833,27.663636,0.0,0.635455,86.675363,1.993636,90.470909,28.591667,0.000,0.606667,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,95.9519


In [0]:
preds = unseen_predictions['Label']
preds.min(),preds.max()

(17.5201, 225.7836)

In [0]:
preds = unseen_predictions['Label']
preds.min(),preds.max()

(15.1003, 229.5528)

In [0]:
d = {'ID': test.ID, 'target': preds}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [0]:
sub.to_csv("catboost_121_norm.csv",index  = False)